# Gif creator

This notebook will create gifs from a series of snapshots. To achieve this you will need to know the following:
1. The central image around which the gif will be created.
2. The deployment ID associated with the images.
3. The country and region of the images.
4. The number of images to include before and after the central image.

In [ ]:
import pandas as pd
import os
from amber_inferences.utils.plotting import gif_creater
import boto3
import json
from boto3.s3.transfer import TransferConfig
from IPython.display import Image, display

## Functions

In [2]:
def get_images_for_gif(central_image, country, deployment, n_before, n_after, data_dir='/gws/nopw/j04/ceh_generic/kgoldmann'):
    """
    Get the images for a gif based on the central image and its context.
    """

    date_str = central_image

    # if starts with 01- or 02- then strip
    if central_image.startswith('01-') or central_image.startswith('02-'):
        date_str = central_image[3:]

    # convert to datetime
    date_str = pd.to_datetime(date_str, format='%Y%m%d%H%M%S-snapshot.jpg')

    # if the hour is after midnight, go to the day before
    if date_str.hour < 12:
        date_str= date_str = date_str - pd.Timedelta(days=1)

    central_image = f'{data_dir}/{country}_inferences_tracking/{central_image}'

    df = pd.read_csv(f'{data_dir}/{country}_inferences_tracking/{deployment}/{deployment}_{date_str.strftime("%Y-%m-%d")}.csv', low_memory=False)

    df['keys'] = df['image_path'].apply(lambda x: f"{deployment}/snapshot_images/{os.path.basename(x)}")

    all_images = sorted(df['image_path'].unique())
    central_image_index = all_images.index(central_image)
    gif_images = all_images[max(0, central_image_index - n_before) : min(central_image_index + n_after + 1, len(all_images))]

    df = df.loc[df['image_path'].isin(gif_images)]

    return df

In [3]:
def initialise_session(credentials_file="credentials.json"):
    """
    Load AWS and API credentials from a configuration file and initialise an AWS session.

    Args:
        credentials_file (str): Path to the credentials JSON file.

    Returns:
        boto3.Client: Initialised S3 client.
    """
    with open(credentials_file, encoding="utf-8") as config_file:
        aws_credentials = json.load(config_file)
    session = boto3.Session(
        aws_access_key_id=aws_credentials["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=aws_credentials["AWS_SECRET_ACCESS_KEY"],
        region_name=aws_credentials["AWS_REGION"],
    )
    client = session.client("s3", endpoint_url=aws_credentials["AWS_URL_ENDPOINT"])
    return client

client = initialise_session('../credentials.json')

# Transfer configuration for optimised S3 download
transfer_config = TransferConfig(
    max_concurrency=20,  # Increase the number of concurrent transfers
    multipart_threshold=8 * 1024 * 1024,  # 8MB
    max_io_queue=1000,
    io_chunksize=262144,  # 256KB
)

def download_images(s3_client, config, key, download_dir, bucket_name):
    download_path = os.path.join(download_dir, os.path.basename(key))
    s3_client.download_file(bucket_name, key, download_path, Config=config)

In [30]:
def get_images(df, dir_path):
    os.makedirs(dir_path, exist_ok=True)

    # remove existing files in the temp directory
    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")

    for i, row in df.iterrows():
        download_images(client, transfer_config, row['keys'], dir_path, region)

def make_gif(input_dir, output_dir, central_image, note):
    # Create a gif from the images in ./temp directory
    gif_output_path = os.path.join(output_dir, f'{note}_{os.path.basename(central_image).split(".")[0].replace("-snapshot", "")}_timelapse.gif')

    # Create the gifs directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create the gif using the gif_creater function
    gif_creater(input_dir, gif_output_path)

    print(f"GIF created successfully at: {gif_output_path}")


## Images of interest

In [28]:
# This is a list of noteworthy images I have curated.
noted_images = [
    ### Thailand dep000074 to dep000078
    {'central_image': '20240813030400-snapshot.jpg', 'deployment': 'dep000077', 'country': 'thailand', 'region': 'tha',
     'n_before': 5, 'n_after': 20, 'note': 'civet cat'},
    {'central_image': '20240906225129-snapshot.jpg', 'deployment': 'dep000075', 'country': 'thailand', 'region': 'tha',
     'n_before': 5, 'n_after': 30, 'note': 'cat on 06/09'},
    {'central_image': '20241003033409-snapshot.jpg', 'deployment': 'dep000075', 'country': 'thailand', 'region': 'tha',
     'n_before': 5, 'n_after': 20, 'note': 'cat on 03/10'},


    # # skip {'central_image': '20241004192419-snapshot.jpg', 'deployment': 'dep000075', 'country': 'thailand', 'region': 'tha',
    # #  'n_before': 5, 'n_after': 50, 'note': 'cat on 04/10'},

    {'central_image': '20241122232709-snapshot.jpg', 'deployment': 'dep000074', 'country': 'thailand', 'region': 'tha',
     'n_before': 5, 'n_after': 5, 'note': 'frog jumping'},
    {'central_image': '20240808001029-snapshot.jpg', 'deployment': 'dep000074', 'country': 'thailand', 'region': 'tha',
     'n_before': 5, 'n_after': 5, 'note': 'frog hunting'},

    ### costa rica (dep000031-40)
    {'central_image': '01-20241121000230-snapshot.jpg', 'deployment': 'dep000032', 'country': 'costarica', 'region': 'cri',
     'n_before': 20, 'n_after': 15, 'note': 'cricket'},
    {'central_image': '01-20240213024750-snapshot.jpg', 'deployment': 'dep000033', 'country': 'costarica', 'region': 'cri',
     'n_before': 20, 'n_after': 20, 'note': 'atlas moth'},
    # skip {'central_image': '01-20240411002540-snapshot.jpg', 'deployment': 'dep000033', 'country': 'costarica', 'region': 'cri',
    # 'n_before': 20, 'n_after': 20, 'note': 'atlas moth2'},
    {'central_image': '20240727003709-snapshot.jpg', 'deployment': 'dep000038', 'country': 'costarica', 'region': 'cri',
     'n_before': 12, 'n_after': 20, 'note': 'white witch'},
    {'central_image': '01-20240210035319-snapshot.jpg', 'deployment': 'dep000034', 'country': 'costarica', 'region': 'cri',
     'n_before': 25, 'n_after': 5, 'note': 'automeric belti eyes'},

    ## singapore dep000045-54
     {'central_image': '01-20240611010749-snapshot.jpg', 'deployment': 'dep000052', 'country': 'singapore', 'region': 'sgp',
     'n_before': 5, 'n_after': 7, 'note': 'rat'},
     {'central_image': '20240907011729-snapshot.jpg', 'deployment': 'dep000050', 'country': 'singapore', 'region': 'sgp',
     'n_before': 10, 'n_after': 7, 'note': 'geckos'},
     {'central_image': '20241007221300-snapshot.jpg', 'deployment': 'dep000050', 'country': 'singapore', 'region': 'sgp',
     'n_before': 15, 'n_after': 10, 'note': 'geckos2'},
     {'central_image': '01-20241128031900-snapshot.jpg', 'deployment': 'dep000045', 'country': 'singapore', 'region': 'sgp',
     'n_before': 15, 'n_after': 15, 'note': 'snail'},
     {'central_image': '01-20241123035219-snapshot.jpg', 'deployment': 'dep000054', 'country': 'singapore', 'region': 'sgp',
     'n_before': 15, 'n_after': 15, 'note': 'snail2'},
     {'central_image': '20250219224710-snapshot.jpg', 'deployment': 'dep000050', 'country': 'singapore', 'region': 'sgp',
     'n_before': 10, 'n_after': 10, 'note': 'four_geckos'},
     {'central_image': '20240531214339-snapshot.jpg', 'deployment': 'dep000050', 'country': 'singapore', 'region': 'sgp',
     'n_before': 5, 'n_after': 15, 'note': 'swallowtail'}


    #  ## kenya
    #  {'central_image': '20241013150339-snapshot.jpg', 'deployment': 'dep000096', 'country': 'kenya', 'region': 'ken',
    #  'n_before': 5, 'n_after': 5, 'note': 'big_moth'}
]

In [27]:
# Loop through all noted images and download
for i in range(len(noted_images)):
    central_image = noted_images[i]['central_image']
    deployment = noted_images[i]['deployment']
    country = noted_images[i]['country']
    region = noted_images[i]['region']
    n_before = noted_images[i]['n_before']
    n_after = noted_images[i]['n_after']
    note = noted_images[i]['note']
    print(f"Processing {i}: {note}")

    try:
        df = get_images_for_gif(central_image, country, deployment, n_before, n_after)
        get_images(df, f'./temp/{note}')
    except Exception as e:
        print(f"Error processing {note}: {deployment}, {e}")

Processing 0: atlas moth2


In [ ]:
# Loop through all noted images and turn the downloaded images into a gif
for i in noted_images:
    note = i['note']

    print(f"Creating GIF for {note}...")

    # Create the GIF
    make_gif(f'./temp/{note}', f'./gifs/{i["country"]}/{i["deployment"]}', i['central_image'], note)

Creating GIF for civet cat...


In [ ]:
# View the gif
gif_output_path = './gifs/thailand/dep000077/'
display(Image(filename=gif_output_path))